##### Models Performance

In [44]:
import pandas as pd 
nodes=pd.read_csv('nodes_df6_tot.csv')

Data preprocessing

In [45]:
nodes.rename(columns={"Colour_GN": "cluster_GN"},inplace=True)

In [46]:
#aggiungiamo a nodes gli hashtag
hashtags=[]
for index,row in nodes.iterrows():
    if row['Colour']=="#b00b69":
        hashtags.append("@EnricoLetta")
    if row['Colour']=="#fb1239":
        hashtags.append("#NoGreenPass")
    if row['Colour']=="#99cccc":
        hashtags.append("@matteosalvinimi")
    if row['Colour']=="#042069":
        hashtags.append("@CasaPoundItalia")
    if row['Colour']=="#007474":
        hashtags.append("#vaccinatevi")

nodes['hashtags']=hashtags

In [51]:
#associo a 1 gli hashtag ritenuti antivax, associo a 2 quelli ritenuti pro vaccino
gruppo=[]

for index,row in nodes.iterrows():
    if row['hashtags']=="#NoGreenPass":
        gruppo.append(1)
    if row['hashtags']=="#vaccinatevi":
        gruppo.append(2)
    if row['hashtags']=="@CasaPoundItalia":
        gruppo.append(1)
    if row['hashtags']=="@matteosalvinimi":
        gruppo.append(1)
    if row['hashtags']=="@EnricoLetta":
        gruppo.append(2)

nodes['gruppo']=gruppo

In [63]:
nodes.to_csv("nodes_definitivo.csv",index=False)

In [53]:
# Prima delle valutazioni è necessario rimuovere gli elementi  esterni al giant component
nodes_eval=nodes[nodes["Giant_Component"]==1]

In [55]:
print(nodes_eval["cluster_eigen"].unique())
print(nodes_eval["cluster_kmeans"].unique())
print(nodes_eval["cluster_GN"].unique())
print(nodes_eval["gruppo"].unique())
# i nomi dei cluster sono standardizzati, possiamo applicare l'evaluation

[1 2]
[1 2]
[1 2]
[1 2]


NB: bisogna invertire i valori dei cluster di cluster_eigen e cluster_kmeans

In [56]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

In [57]:
class Evaluations:

    def __init__(self, df, clusters_col_name):
        self.df=df
        self.clusters_col_name=clusters_col_name

    def conf_matrix(self):
        conf_mat=confusion_matrix(list(self.df.gruppo), list(self.df[self.clusters_col_name]))
        df_conf_norm = pd.DataFrame(conf_mat)
        df_conf_norm_r=pd.DataFrame(conf_mat/np.array( [conf_mat.sum(axis=1),conf_mat.sum(axis=1),conf_mat.sum(axis=1)]).T)
        plt.matshow(df_conf_norm_r, cmap='Reds')
        plt.colorbar()

        for (i, j), z in np.ndenumerate(df_conf_norm.to_numpy()):
            plt.text(j, i, '{:0.0f}'.format(z), ha='center', va='center')

        plt.xlabel('Predicted')
        plt.ylabel('Actual')

        plt.show()

    def metric(self):
        m=metrics.classification_report(list(self.df.gruppo), list(self.df[self.clusters_col_name]), digits=3)
        print(m)

##### Eigen Evaluation

In [58]:
eval_eigen=Evaluations(nodes_eval,"cluster_eigen")
eval_eigen.metric()

              precision    recall  f1-score   support

           1      0.478     0.647     0.550      3010
           2      0.048     0.025     0.033      2180

    accuracy                          0.386      5190
   macro avg      0.263     0.336     0.291      5190
weighted avg      0.298     0.386     0.333      5190



In [60]:
eval_kmeans=Evaluations(nodes_eval,"cluster_kmeans")
eval_kmeans.metric()

              precision    recall  f1-score   support

           1      0.514     0.758     0.612      3010
           2      0.025     0.009     0.013      2180

    accuracy                          0.443      5190
   macro avg      0.270     0.383     0.313      5190
weighted avg      0.309     0.443     0.361      5190



In [61]:
eval_kmeans=Evaluations(nodes_eval,"cluster_GN")
eval_kmeans.metric()

              precision    recall  f1-score   support

           1      0.911     0.961     0.935      3010
           2      0.941     0.870     0.904      2180

    accuracy                          0.923      5190
   macro avg      0.926     0.915     0.920      5190
weighted avg      0.924     0.923     0.922      5190

